# RNN Hello World

RNN을 사용해서 "Hello Anderson Hi World!. Everything is awesome!" 이라는 글자들을 예측하는 RNN을 만들어봅니다. 

In [1]:
import numpy as np
import tensorflow as tf

from keras.layers import LSTM, SimpleRNN, GRU
from keras.models import Sequential

Using TensorFlow backend.


## Data

In [2]:
LABEL_SIZE = 100
TRAINING_SIZE = 50000
TEST_SIZE = 10000

def create_data(size=50000, maximum=1000, end_token=2):
    max_length = len(bin(maximum-1)[2:]) + 1
    rands = np.random.randint(0, maximum, size=500000)
    xs = np.zeros((size, 1, max_length), dtype='float32')
    ys = np.zeros((size, 1, maximum), dtype='int32')
    for i in range(size):
        x = np.zeros(20, dtype='float32')
        rand_bin = bin(rands[i])[2:]+ str(end_token)
        rand_bin = list(map(float, list(rand_bin)))
        xs[i, 0, :len(rand_bin)] = rand_bin    
        ys[i, 0, rands[i]] = 1
    
    return xs, ys

train_x, train_y = create_data(size=TRAINING_SIZE, maximum=LABEL_SIZE)
test_x, test_y = create_data(size=TEST_SIZE, maximum=LABEL_SIZE)

SENTENCE_LENGTH = train_x.shape[2]

# LSTM Model

1. categorical_crossentropy 가 작동하지 않음 (자세한 분석 필요)

In [14]:
lstm = Sequential(name='LSTM')
lstm.add(LSTM(output_dim=LABEL_SIZE, batch_input_shape=(None, None, SENTENCE_LENGTH), 
               return_sequences=True, name='lstm'))
lstm.compile(loss='mean_squared_error', optimizer='adam')
lstm.fit(train_x, train_y, verbose=2, nb_epoch=5)

Epoch 1/5
8s - loss: 0.0087
Epoch 2/5
8s - loss: 0.0059
Epoch 3/5
8s - loss: 0.0046
Epoch 4/5
8s - loss: 0.0040
Epoch 5/5
8s - loss: 0.0036


In [15]:
lstm.evaluate(test_x, test_y, verbose=2)

0.0034724433556199072

In [16]:
def evaluate(model, x, y):
    y_preds = model.predict(x)
    n_correct = 0
    for i in range(TEST_SIZE):
        y_pred = np.argmax(y_preds[i, 0])
        y_true = np.argmax(y[i, 0])
        if y_pred == y_true:
            n_correct += 1
            
    print(f'테스트 갯수: {TEST_SIZE}, 맞은 갯수: {n_correct}')
    print('accuracy:', n_correct/float(TEST_SIZE))
    
evaluate(lstm, test_x, test_y)

테스트 갯수: 10000, 맞은 갯수: 10000
accuracy: 1.0


## RNN MODEL


In [6]:
train_rnn_y = train_y.reshape((train_y.shape[0],train_y.shape[2]))
test_rnn_y = test_y.reshape((test_y.shape[0],test_y.shape[2]))

print('train_x shape:', train_x.shape)
print('train_y shape:', test_rnn_y.shape)

train_x shape: (50000, 1, 8)
train_y shape: (10000, 100)


In [7]:
rnn = Sequential(name='RNN')
rnn.add(SimpleRNN(output_dim=LABEL_SIZE, name='rnn', batch_input_shape=(None, None, SENTENCE_LENGTH)))
rnn.compile(loss='mean_squared_error', optimizer='adam')
rnn.fit(train_x, train_rnn_y, verbose=2, nb_epoch=5)

Epoch 1/5
3s - loss: 0.0141
Epoch 2/5
3s - loss: 0.0092
Epoch 3/5
3s - loss: 0.0092
Epoch 4/5
3s - loss: 0.0092
Epoch 5/5
3s - loss: 0.0092


In [8]:
evaluate(rnn, test_x, test_rnn_y)

테스트 갯수: 10000, 맞은 갯수: 10000
accuracy: 1.0


## GRU Model

In [10]:
gru = Sequential(name='RNN')
gru.add(GRU(output_dim=LABEL_SIZE, name='rnn', batch_input_shape=(None, None, SENTENCE_LENGTH)))
gru.compile(loss='mean_squared_error', optimizer='adam')
gru.fit(train_x, train_rnn_y, verbose=2, nb_epoch=5)

Epoch 1/5
7s - loss: 0.0102
Epoch 2/5
7s - loss: 0.0075
Epoch 3/5
6s - loss: 0.0059
Epoch 4/5
7s - loss: 0.0049
Epoch 5/5
7s - loss: 0.0042


In [11]:
evaluate(gru, test_x, test_rnn_y)

테스트 갯수: 10000, 맞은 갯수: 10000
accuracy: 1.0
